# Interior point methods

In [17]:
using LinearAlgebra
using SparseArrays

# local modules
push!(LOAD_PATH, realpath("../code"))
using lp_problem
using lp_read_mps_mip

In [22]:
function read_file_to_string(filepath)
    mps_string = open(filepath, "r") do f
        read(f, String)
    end
    return mps_string
end

read_file_to_string (generic function with 1 method)

In [24]:
filepath ="../../check/benchmarks/mps_files/test.mps"
println(read_file_to_string(filepath))

NAME          BLEND

OBJSENSE
 MAX
 
ROWS
 N  PROF
 L  VVEG
 L  NVEG
 L  UHRD
 G  LHRD
 E  CONT
COLUMNS
    VEG01     PROF      -110.0    VVEG         1.0
    VEG01     UHRD         8.8    LHRD         8.8
    VEG01     CONT         1.0
    VEG02     PROF      -120.0    VVEG         1.0
    VEG02     UHRD         6.1    LHRD         6.1
    VEG02     CONT         1.0
    OIL01     PROF      -130.0    NVEG         1.0
    OIL01     UHRD         2.0    LHRD         2.0
    OIL01     CONT         1.0
    OIL02     PROF      -110.0    NVEG         1.0
    OIL02     UHRD         4.2    LHRD         4.2
    OIL02     CONT         1.0
    OIL03     PROF      -115.0    NVEG         1.0
    OIL03     UHRD         5.0    LHRD         5.0
    OIL03     CONT         1.0
    PROD      PROF       150.0    UHRD        -6.0
    PROD      LHRD        -3.0    CONT        -1.0
RHS
    RHS       VVEG       200.0
    RHS       NVEG       250.0
    RHS       UHRD         0.0
    RHS       LHRD         0.0
 

In [11]:
lp = read_mps_from_file("../../benchmarks/mps_files/test.mps")

MIPProblem(false, [-110.0, -120.0, -130.0, -110.0, -115.0, 150.0], sparse([1, 3, 4, 5, 1, 3, 4, 5, 2, 3  …  3, 4, 5, 2, 3, 4, 5, 3, 4, 5], [1, 1, 1, 1, 2, 2, 2, 2, 3, 3  …  4, 4, 4, 5, 5, 5, 5, 6, 6, 6], [1.0, 8.8, -8.8, 1.0, 1.0, 6.1, -6.1, 1.0, 1.0, 2.0  …  4.2, -4.2, 1.0, 1.0, 5.0, -5.0, 1.0, -6.0, 3.0, -1.0], 5, 6), [200.0, 250.0, 0.0, -0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [Inf, Inf, Inf, Inf, Inf, Inf], ["VEG01", "VEG02", "OIL01", "OIL02", "OIL03", "PROD"], [:Continuous, :Continuous, :Continuous, :Continuous, :Continuous, :Continuous], ['L', 'L', 'L', 'G', 'E'])

In [15]:


# Interior Point Method for solving LPProblem
function interior_point_solver(lp::MIPProblem; tol=1e-6, max_iter=100)
    # Unpack problem data
    c = lp.is_minimize ? lp.c : -lp.c  # Adjust for maximization problems
    A = lp.A
    b = lp.b
    m, n = size(A)

    # Initialize variables
    x = ones(n)  # Primal variables (start with feasible guess)
    s = ones(n)  # Slack variables
    y = zeros(m)  # Dual variables
    μ = 1.0  # Barrier parameter

    for iter in 1:max_iter
        # Compute residuals
        r_dual = A' * y + s - c
        r_cent = s .* x - μ * ones(n)
        r_pri = A * x - b

        # Check convergence
        if norm(r_dual, Inf) < tol && norm(r_pri, Inf) < tol && norm(r_cent, Inf) < tol
            println("Converged in $iter iterations.")
            return x
        end

        # Form the KKT matrix
        D_inv = Diagonal(1.0 ./ x)
        M = A * D_inv * A'
        rhs = -(r_pri + A * D_inv * (r_cent - D_inv * r_dual))

        # Solve for the dual variables
        Δy = M \ rhs

        # Solve for the primal and slack variables
        Δs = r_dual - A' * Δy
        Δx = -x .+ D_inv * (r_cent - Δs .* x)

        # Line search for feasibility
        α = 1.0
        for i in 1:n
            if Δx[i] < 0
                α = min(α, -x[i] / Δx[i])
            end
            if Δs[i] < 0
                α = min(α, -s[i] / Δs[i])
            end
        end
        α = 0.99 * α  # Take a step just inside the feasible region

        # Update variables
        x += α * Δx
        s += α * Δs
        y += α * Δy

        # Update the barrier parameter
        μ *= 0.9  # Reduce barrier parameter
    end

    println("Maximum iterations reached without convergence.")
    return x
end


interior_point_solver (generic function with 2 methods)

In [16]:
# # Example usage
# A = sparse([1.0 2.0; 3.0 4.0])
# b = [1.0, 1.0]
# c = [-1.0, -2.0]
# l = [0.0, 0.0]
# u = [Inf, Inf]
# vars = ["x1", "x2"]
# variable_types = [:Continuous, :Continuous]
# constraint_types = ['L', 'L']

# lp = LPProblem(true, c, A, b, l, u, vars, variable_types, constraint_types)

# Solve the LP problem
solution = interior_point_solver(lp)
println("Optimal solution: ", solution)


Maximum iterations reached without convergence.
Optimal solution: [0.6449107699107709, 1.7394068374694456, 1.9575822029873267, 0.9862445276585994, 0.8123908145697386, 1.9963235303711915]


In [ ]:
# Solve the LP problem
solution = interior_point_solver(lp, tol=1e-8, max_iter=5000)
println("Optimal solution: ", solution)

# References

Karmarkar, N. A new polynomial-time algorithm for linear programming. Combinatorica 4, 373–395 (1984). https://doi.org/10.1007/BF02579150


```bibtex
@book{SIAMB0000215,
 title={Primal-dual interior-point methods},
 author={Wright, Stephen J.},
 year={1997},
 publisher={Society for Industrial and Applied Mathematics},
 pages={310},
 isbn={9780898713824},
 url={https://portal-igpublish-com.ezproxy.newcastle.edu.au/iglibrary/search/SIAMB0000215.html}
}
```

Megiddo, N. On the complexity of computing an approximate solution for combinatorial optimization problems. SIAM Journal on Computing 18, 1189-1198 (1989). https://doi.org/10.1137/0218074

Forsgren, A., & Gill, P. E. (2007). A Matlab Implementation of a Primal-Dual Interior Point Method for Linear Programming. Computational Optimization and Applications, 36(2-3), 209-241. https://doi.org/10.1007/s10589-006-9005-9